### CSE 4705 Project: LLM Powered Analysis

Tom McCarthy and Filip Graham

## Downloading Unstructured Files

We downloaded Connecticut municipal profiles, which are 2-page pdf summaries with various statistics on municipalities. Below is sample code we used to download files. In total there were around 1000 files. There were slight differences in the URL structure depending on the year, which is why we have slightly different versions for each year.


In [ ]:
from unstructured.partition.pdf import partition_pdf
import requests
import time
import openai
import os
import json
import re

In [ ]:
mun_2016 = ['andover-2016','ansonia-2016','ashford-2016','avon-2016','barkhamsted-2016','beacon-falls-2016','berlin-2016','bethany-2016','bethel-2016','bethlehem-2016','bloomfield-2016','bolton-2016','bozrah-2016','branford-2016','bridgeport-2016','bridgewater-2016','bristol-2016','brookfield-2016','brooklyn-2016','burlington-2016','canaan-2016','canterbury-2016','canton-2016','chaplin-2016','cheshire-2016','chester-2016','clinton-2016','colchester-2016','colebrook-2016','columbia-2016','cornwall-2016','coventry-2016','cromwell-2016','danbury-2016','darien-2016','deep-river-2016','derby-2016','durham-2016','east-granby-2016','east-haddam-2016','east-hampton-2016','east-hartford-2016','east-haven-2016','east-lyme-2016','east-windsor-2016','eastford-2016','easton-2016','ellington-2016','enfield-2016','essex-2016','fairfield-2016','farmington-2016','franklin-2016','glastonbury-2016','goshen-2016','granby-2016','greenwich-2016','griswold-2016','groton-2016','guilford-2016','haddam-2016','hamden-2016','hampton-2016','hartford-2016','hartland-2016','harwinton-2016','hebron-2016','kent-2016','killingly-2016','killingworth-2016','lebanon-2016','ledyard-2016','lisbon-2016','litchfield-2016','lyme-2016','madison-2016','manchester-2016','mansfield-2016','marlborough-2016','meriden-2016','middlebury-2016','middlefield-2016','middletown-2016','milford-2016','monroe-2016','montville-2016','morris-2016','naugatuck-2016','new-britain-2016','new-canaan-2016','new-fairfield-2016','new-hartford-2016','new-haven-2016','new-london-2016','new-milford-2016','newington-2016','newtown-2016','norfolk-2016','north-branford-2016','north-canaan-2016','north-haven-2016','north-stonington-2016','norwalk-2016','norwich-2016','old-lyme-2016','old-saybrook-2016','orange-2016','oxford-2016','plainfield-2016','plainville-2016','plymouth-2016','pomfret-2016','portland-2016','preston-2016','prospect-2016','putnam-2016','redding-2016','ridgefield-2016','rocky-hill-2016','roxbury-2016','salem-2016','salisbury-2016','scotland-2016','seymour-2016','sharon-2016','shelton-2016','sherman-2016','simsbury-2016','somers-2016','south-windsor-2016','southbury-2016','southington-2016','sprague-2016','stafford-2016','stamford-2016','sterling-2016','stonington-2016','stratford-2016','suffield-2016','thomaston-2016','thompson-2016','tolland-2016','torrington-2016','trumbull-2016','union-2016','vernon-2016','voluntown-2016','wallingford-2016','warren-2016','washington-2016','waterbury-2016','waterford-2016','watertown-2016','west-hartford-2016','west-haven-2016','westbrook-2016','weston-2016','westport-2016','wethersfield-2016','willington-2016','wilton-2016','winchester-2016','windham-2016','windsor-2016','windsor-locks-2016','wolcott-2016','woodbridge-2016','woodbury-2016','woodstock-2016']
mun_2017 = ['andover-2017','ansonia-2017','ashford-2017','avon-2017','barkhamsted-2017','beacon-falls-2017','berlin-2017','bethany-2017','bethel-2017','bethlehem-2017','bloomfield-2017','bolton-2017','bozrah-2017','branford-2017','bridgeport-2017','bridgewater-2017','bristol-2017','brookfield-2017','brooklyn-2017','burlington-2017','canaan-2017','canterbury-2017','canton-2017','chaplin-2017','cheshire-2017','chester-2017','clinton-2017','colchester-2017','colebrook-2017','columbia-2017','cornwall-2017','coventry-2017','cromwell-2017','danbury-2017','darien-2017','deep-river-2017','derby-2017','durham-2017','east-granby-2017','east-haddam-2017','east-hampton-2017','east-hartford-2017','east-haven-2017','east-lyme-2017','east-windsor-2017','eastford-2017','easton-2017','ellington-2017','enfield-2017','essex-2017','fairfield-2017','farmington-2017','franklin-2017','glastonbury-2017','goshen-2017','granby-2017','greenwich-2017','griswold-2017','groton-2017','guilford-2017','haddam-2017','hamden-2017','hampton-2017','hartford-2017','hartland-2017','harwinton-2017','hebron-2017','kent-2017','killingly-2017','killingworth-2017','lebanon-2017','ledyard-2017','lisbon-2017','litchfield-2017','lyme-2017','madison-2017','manchester-2017','mansfield-2017','marlborough-2017','meriden-2017','middlebury-2017','middlefield-2017','middletown-2017','milford-2017','monroe-2017','montville-2017','morris-2017','naugatuck-2017','new-britain-2017','new-canaan-2017','new-fairfield-2017','new-hartford-2017','new-haven-2017','new-london-2017','new-milford-2017','newington-2017','newtown-2017','norfolk-2017','north-branford-2017','north-canaan-2017','north-haven-2017','north-stonington-2017','norwalk-2017','norwich-2017','old-lyme-2017','old-saybrook-2017','orange-2017','oxford-2017','plainfield-2017','plainville-2017','plymouth-2017','pomfret-2017','portland-2017','preston-2017','prospect-2017','putnam-2017','redding-2017','ridgefield-2017','rocky-hill-2017','roxbury-2017','salem-2017','salisbury-2017','scotland-2017','seymour-2017','sharon-2017','shelton-2017','sherman-2017','simsbury-2017','somers-2017','south-windsor-2017','southbury-2017','southington-2017','sprague-2017','stafford-2017','stamford-2017','sterling-2017','stonington-2017','stratford-2017','suffield-2017','thomaston-2017','thompson-2017','tolland-2017','torrington-2017','trumbull-2017','union-2017','vernon-2017','voluntown-2017','wallingford-2017','warren-2017','washington-2017','waterbury-2017','waterford-2017','watertown-2017','west-hartford-2017','west-haven-2017','westbrook-2017','weston-2017','westport-2017','wethersfield-2017','willington-2017','wilton-2017','winchester-2017','windham-2017','windsor-2017','windsor-locks-2017','wolcott-2017','woodbridge-2017','woodbury-2017','woodstock-2017']
mun_2018 = ['andover-2018','ansonia-2018','ashford-2018','avon-2018','barkhamsted-2018','beacon-falls-2018','berlin-2018','bethany-2018','bethel-2018','bethlehem-2018','bloomfield-2018','bolton-2018','bozrah-2018','branford-2018','bridgeport-2018','bridgewater-2018','bristol-2018','brookfield-2018','brooklyn-2018','burlington-2018','canaan-2018','canterbury-2018','canton-2018','chaplin-2018','cheshire-2018','chester-2018','clinton-2018','colchester-2018','colebrook-2018','columbia-2018','cornwall-2018','coventry-2018','cromwell-2018','danbury-2018','darien-2018','deep-river-2018','derby-2018','durham-2018','east-granby-2018','east-haddam-2018','east-hampton-2018','east-hartford-2018','east-haven-2018','east-lyme-2018','east-windsor-2018','eastford-2018','easton-2018','ellington-2018','enfield-2018','essex-2018','fairfield-2018','farmington-2018','franklin-2018','glastonbury-2018','goshen-2018','granby-2018','greenwich-2018','griswold-2018','groton-2018','guilford-2018','haddam-2018','hamden-2018','hampton-2018','hartford-2018','hartland-2018','harwinton-2018','hebron-2018','kent-2018','killingly-2018','killingworth-2018','lebanon-2018','ledyard-2018','lisbon-2018','litchfield-2018','lyme-2018','madison-2018','manchester-2018','mansfield-2018','marlborough-2018','meriden-2018','middlebury-2018','middlefield-2018','middletown-2018','milford-2018','monroe-2018','montville-2018','morris-2018','naugatuck-2018','new-britain-2018','new-canaan-2018','new-fairfield-2018','new-hartford-2018','new-haven-2018','new-london-2018','new-milford-2018','newington-2018','newtown-2018','norfolk-2018','north-branford-2018','north-canaan-2018','north-haven-2018','north-stonington-2018','norwalk-2018','norwich-2018','old-lyme-2018','old-saybrook-2018','orange-2018','oxford-2018','plainfield-2018','plainville-2018','plymouth-2018','pomfret-2018','portland-2018','preston-2018','prospect-2018','putnam-2018','redding-2018','ridgefield-2018','rocky-hill-2018','roxbury-2018','salem-2018','salisbury-2018','scotland-2018','seymour-2018','sharon-2018','shelton-2018','sherman-2018','simsbury-2018','somers-2018','south-windsor-2018','southbury-2018','southington-2018','sprague-2018','stafford-2018','stamford-2018','sterling-2018','stonington-2018','stratford-2018','suffield-2018','thomaston-2018','thompson-2018','tolland-2018','torrington-2018','trumbull-2018','union-2018','vernon-2018','voluntown-2018','wallingford-2018','warren-2018','washington-2018','waterbury-2018','waterford-2018','watertown-2018','west-hartford-2018','west-haven-2018','westbrook-2018','weston-2018','westport-2018','wethersfield-2018','willington-2018','wilton-2018','winchester-2018','windham-2018','windsor-2018','windsor-locks-2018','wolcott-2018','woodbridge-2018','woodbury-2018','woodstock-2018']
mun_2019 = ['andover-2019','ansonia-2019','ashford-2019','avon-2019','barkhamsted-2019','beacon-falls-2019','berlin-2019','bethany-2019','bethel-2019','bethlehem-2019','bloomfield-2019','bolton-2019','bozrah-2019','branford-2019','bridgeport-2019','bridgewater-2019','bristol-2019','brookfield-2019','brooklyn-2019','burlington-2019','canaan-2019','canterbury-2019','canton-2019','chaplin-2019','cheshire-2019','chester-2019','clinton-2019','colchester-2019','colebrook-2019','columbia-2019','cornwall-2019','coventry-2019','cromwell-2019','danbury-2019','darien-2019','deep-river-2019','derby-2019','durham-2019','east-granby-2019','east-haddam-2019','east-hampton-2019','east-hartford-2019','east-haven-2019','east-lyme-2019','east-windsor-2019','eastford-2019','easton-2019','ellington-2019','enfield-2019','essex-2019','fairfield-2019','farmington-2019','franklin-2019','glastonbury-2019','goshen-2019','granby-2019','greenwich-2019','griswold-2019','groton-2019','guilford-2019','haddam-2019','hamden-2019','hampton-2019','hartford-2019','hartland-2019','harwinton-2019','hebron-2019','kent-2019','killingly-2019','killingworth-2019','lebanon-2019','ledyard-2019','lisbon-2019','litchfield-2019','lyme-2019','madison-2019','manchester-2019','mansfield-2019','marlborough-2019','meriden-2019','middlebury-2019','middlefield-2019','middletown-2019','milford-2019','monroe-2019','montville-2019','morris-2019','naugatuck-2019','new-britain-2019','new-canaan-2019','new-fairfield-2019','new-hartford-2019','new-haven-2019','new-london-2019','new-milford-2019','newington-2019','newtown-2019','norfolk-2019','north-branford-2019','north-canaan-2019','north-haven-2019','north-stonington-2019','norwalk-2019','norwich-2019','old-lyme-2019','old-saybrook-2019','orange-2019','oxford-2019','plainfield-2019','plainville-2019','plymouth-2019','pomfret-2019','portland-2019','preston-2019','prospect-2019','putnam-2019','redding-2019','ridgefield-2019','rocky-hill-2019','roxbury-2019','salem-2019','salisbury-2019','scotland-2019','seymour-2019','sharon-2019','shelton-2019','sherman-2019','simsbury-2019','somers-2019','south-windsor-2019','southbury-2019','southington-2019','sprague-2019','stafford-2019','stamford-2019','sterling-2019','stonington-2019','stratford-2019','suffield-2019','thomaston-2019','thompson-2019','tolland-2019','torrington-2019','trumbull-2019','union-2019','vernon-2019','voluntown-2019','wallingford-2019','warren-2019','washington-2019','waterbury-2019','waterford-2019','watertown-2019','west-hartford-2019','west-haven-2019','westbrook-2019','weston-2019','westport-2019','wethersfield-2019','willington-2019','wilton-2019','winchester-2019','windham-2019','windsor-2019','windsor-locks-2019','wolcott-2019','woodbridge-2019','woodbury-2019','woodstock-2019']
mun_2021 = ['Andover','Ansonia','Ashford','Avon','Barkhamsted','Beacon-Falls','Berlin','Bethany','Bethel','Bethlehem','Bloomfield','Bolton','Bozrah','Branford','Bridgeport','Bridgewater','Bristol','Brookfield','Brooklyn','Burlington','Canaan','Canterbury','Canton','Chaplin','Cheshire','Chester','Clinton','Colchester','Colebrook','Columbia','Cornwall','Coventry','Cromwell','Danbury','Darien','Deep-River','Derby','Durham','East-Granby','East-Haddam','East-Hampton','East-Hartford','East-Haven','East-Lyme','East-Windsor','Eastford','Easton','Ellington','Enfield','Essex','Fairfield','Farmington','Franklin','Glastonbury','Goshen','Granby','Greenwich','Griswold','Groton','Guilford','Haddam','Hamden','Hampton','Hartford','Hartland','Harwinton','Hebron','Kent','Killingly','Killingworth','Lebanon','Ledyard','Lisbon','Litchfield','Lyme','Madison','Manchester','Mansfield','Marlborough','Meriden','Middlebury','Middlefield','Middletown','Milford','Monroe','Montville','Morris','Naugatuck','New-Britain','New-Canaan','New-Fairfield','New-Hartford','New-Haven','New-London','New-Milford','Newington','Newtown','Norfolk','North-Branford','North-Canaan','North-Haven','North-Stonington','Norwalk','Norwich','Old-Lyme','Old-Saybrook','Orange','Oxford','Plainfield','Plainville','Plymouth','Pomfret','Portland','Preston','Prospect','Putnam','Redding','Ridgefield','Rocky-Hill','Roxbury','Salem','Salisbury','Scotland','Seymour','Sharon','Shelton','Sherman','Simsbury','Somers','South-Windsor','Southbury','Southington','Sprague','Stafford','Stamford','Sterling','Stonington','Stratford','Suffield','Thomaston','Thompson','Tolland','Torrington','Trumbull','Union','Vernon','Voluntown','Wallingford','Warren','Washington','Waterbury','Waterford','Watertown','West-Hartford','West-Haven','Westbrook','Weston','Westport','Wethersfield','Willington','Wilton','Winchester','Windham','Windsor','Windsor-Locks','Wolcott','Woodbridge','Woodbury','Woodstock']
mun_2023 = ['Andover','Ansonia','Ashford','Avon','Barkhamsted','Beacon-Falls','Berlin','Bethany','Bethel','Bethlehem','Bloomfield','Bolton','Bozrah','Branford','Bridgeport','Bridgewater','Bristol','Brookfield','Brooklyn','Burlington','Canaan-FallsVillage','Canterbury','Canton','Chaplin','Cheshire','Chester','Clinton','Colchester','Colebrook','Columbia','Cornwall','Coventry','Cromwell','Danbury','Darien','Deep-River','Derby','Durham','East-Granby','East-Haddam','East-Hampton','East-Hartford','East-Haven','East-Lyme','East-Windsor','Eastford','Easton','Ellington','Enfield','Essex','Fairfield','Farmington','Franklin','Glastonbury','Goshen','Granby','Greenwich','Griswold','Groton','Guilford','Haddam','Hamden','Hampton','Hartford','Hartland','Harwinton','Hebron','Kent','Killingly','Killingworth','Lebanon','Ledyard','Lisbon','Litchfield','Lyme','Madison','Manchester','Mansfield','Marlborough','Meriden','Middlebury','Middlefield','Middletown','Milford','Monroe','Montville','Morris','Naugatuck','New-Britain','New-Canaan','New-Fairfield','New-Hartford','New-Haven','New-London','New-Milford','Newington','Newtown','Norfolk','North-Branford','North-Canaan','North-Haven','North-Stonington','Norwalk','Norwich','Old-Lyme','Old-Saybrook','Orange','Oxford','Plainfield','Plainville','Plymouth','Pomfret','Portland','Preston','Prospect','Putnam','Redding','Ridgefield','Rocky-Hill','Roxbury','Salem','Salisbury','Scotland','Seymour','Sharon','Shelton','Sherman','Simsbury','Somers','South-Windsor','Southbury','Southington','Sprague','Stafford','Stamford','Sterling','Stonington','Stratford','Suffield','Thomaston','Thompson','Tolland','Torrington','Trumbull','Union','Vernon','Voluntown','Wallingford','Warren','Washington','Waterbury','Waterford','Watertown','West-Hartford','West-Haven','Westbrook','Weston','Westport','Wethersfield','Willington','Wilton','Winchester','Windham','Windsor','Windsor-Locks','Wolcott','Woodbridge','Woodbury','Woodstock']
for mun in mun_2023:
    with open(r'pdfs\{}-2023.pdf'.format(mun.title()), 'wb') as f:
        response = requests.get(r'https://s3-us-west-2.amazonaws.com/cerc-pdfs/2023/{}.pdf'.format(mun))
        response.raise_for_status()
        f.write(response.content)

## Parsing the Unstructured Files

We attempted to use the Unstructured.io python library to parse the PDFs, but it did not handle the parsing well. We included one of the PDFs to this project. It has multiple columns and tables that are not clearly defined using boundaries, so this is probably why. The output contains a jumble of multiple tables out of order, making it extremely difficult to use effectively.

In [ ]:
elements = partition_pdf("Andover-2016.pdf", strategy = 'hi_res')
for element in elements:
    print(element.text)

## Using OpenAI/ChatGPT API
We tried parsing one of the files using ChatGPT and prompted for specific attributes we want to include in our dataset. It worked well, so we decided to develop a script that automates the prompting and retrieval process. We store the outputs in JSON files and have included a sample JSON response file in the project.

In [ ]:
os.environ["OPENAI_API_KEY"] = ""
client = openai.OpenAI()

In [ ]:
# years_test = ['2016']
# mun_test = ['Norwalk']
muns = ['Andover','Ansonia','Ashford','Avon','Barkhamsted','Beacon-Falls','Berlin','Bethany','Bethel','Bethlehem','Bloomfield','Bolton','Bozrah','Branford','Bridgeport','Bridgewater','Bristol','Brookfield','Brooklyn','Burlington','Canaan','Canterbury','Canton','Chaplin','Cheshire','Chester','Clinton','Colchester','Colebrook','Columbia','Cornwall','Coventry','Cromwell','Danbury','Darien','Deep-River','Derby','Durham','East-Granby','East-Haddam','East-Hampton','East-Hartford','East-Haven','East-Lyme','East-Windsor','Eastford','Easton','Ellington','Enfield','Essex','Fairfield','Farmington','Franklin','Glastonbury','Goshen','Granby','Greenwich','Griswold','Groton','Guilford','Haddam','Hamden','Hampton','Hartford','Hartland','Harwinton','Hebron','Kent','Killingly','Killingworth','Lebanon','Ledyard','Lisbon','Litchfield','Lyme','Madison','Manchester','Mansfield','Marlborough','Meriden','Middlebury','Middlefield','Middletown','Milford','Monroe','Montville','Morris','Naugatuck','New-Britain','New-Canaan','New-Fairfield','New-Hartford','New-Haven','New-London','New-Milford','Newington','Newtown','Norfolk','North-Branford','North-Canaan','North-Haven','North-Stonington','Norwalk','Norwich','Old-Lyme','Old-Saybrook','Orange','Oxford','Plainfield','Plainville','Plymouth','Pomfret','Portland','Preston','Prospect','Putnam','Redding','Ridgefield','Rocky-Hill','Roxbury','Salem','Salisbury','Scotland','Seymour','Sharon','Shelton','Sherman','Simsbury','Somers','South-Windsor','Southbury','Southington','Sprague','Stafford','Stamford','Sterling','Stonington','Stratford','Suffield','Thomaston','Thompson','Tolland','Torrington','Trumbull','Union','Vernon','Voluntown','Wallingford','Warren','Washington','Waterbury','Waterford','Watertown','West-Hartford','West-Haven','Westbrook','Weston','Westport','Wethersfield','Willington','Wilton','Winchester','Windham','Windsor','Windsor-Locks','Wolcott','Woodbridge','Woodbury','Woodstock']
# years = ["2016", "2017", "2018", "2019", "2021", "2023"]
years = ['2016']

attributes = {
    '2016': {
        "Population: 2010-2014: Town",
        "Race/Ethnicity: White: Town", 
        "Race/Ethnicity: Black: Town",
        "Race/Ethnicity: Asian: Town",
        "Race/Ethnicity: Other: Town",
        "Race/Ethnicity: Hispanic: Town",
        "Median Age: Town",
        "Med. HH Inc.: Town",
        "Poverty Rate: Town",
        "Educational Attainment: High School Graduate: Town",
        "Educational Attainment: Associates Degree: Town",
        "Educational Attainment: Bachelors or Higher: Town",
        "Actual Mill Rate",
        "Labor Force: Unemployment Rate: Town",
        "Crime Rate: Per 100000 residents: Town"
    }
}

for year in years:
    for mun in muns:
        # with open(f"/pdfs/{mun}-{year}.pdf", "rb") as file: 
        file = client.files.create(
            # file=open("Norwalk-2016.pdf", "rb"),
            # file=file,
            file=open(f"pdfs/{mun}-{year}.pdf", "rb"),
            purpose="user_data"
        )

        response = client.responses.create(
            model="gpt-4o",
            input=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "input_file",
                            "file_id": file.id,
                        },
                        {
                            "type": "input_text",
                            "text": f"You are an expert extraction algorithm. Only extract relevant information from the pdf. If you do not know the value of an attribute asked to extract, return null for the attribute's value. Attributes: {attributes[year]}. Return JSON markdown only, I just need JSON markdown. Do not reply with anything else",
                        },
                    ]
                }
            ]
        )
        
        # create the output file path
        output_file_path = os.path.join("outputs_in_json", f"{mun}-{year}.json")

        # save the response to a JSON file
        with open(output_file_path, "w", encoding="utf-8") as json_file:
            # remove the markdown code block
            json_string = re.sub(r"^```json\n|\n```$", "", response.output_text)

            print('ORIGINAL OUTPUT FROM OPENAI')
            print(response.output_text)
            print()
            # parse the JSON
            data = json.loads(json_string)
            # print(data)
            
            json.dump(data, json_file, indent=4)

        print(f"Saved response to {output_file_path}")

        # print(response.output_text)